<a href="https://colab.research.google.com/github/ShadowF1endZ/Recommender-System-for-Tourism/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain-ollama
!pip install -q langchain
!pip install -q langchain_community
!pip install -q sentence-transformers
!pip install -qU langchain-google-genai

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from sentence_transformers import SentenceTransformer
from langchain_google_genai import ChatGoogleGenerativeAI
import re
import json
import subprocess
import time
import threading
import psycopg2

In [ ]:
import getpass
import os

postgres_url = getpass.getpass("Enter your postgresql url: ")

In [ ]:
model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')

In [ ]:
!curl https://ollama.ai/install.sh | sh

In [ ]:
# Start the ollama server in a new process
process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Function to print server output
def print_output(process):
    while True:
        output = process.stdout.readline()
        if output == b'' and process.poll() is not None:
            break
        if output:
            print(output.strip().decode('utf-8'))
        time.sleep(1)

# Start a thread to print server output
thread = threading.Thread(target=print_output, args=(process,))
thread.start()

print("Ollama server is running in the background")

In [ ]:
!ollama pull llama3:latest

In [ ]:
llm = OllamaLLM(model="llama3:latest")

In [ ]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
template = """
    You are an AI travel suggestion chatbot. Your task is to analyze the Request:

    Request: "{travel_request}"

    Extract the following 7 pieces of information not in any specific order: Type, District, City, Number_of_people, Price, Rating, and Description.
    Return the result by vietnamese in the following format:
    {{
        "Type": "...",
        "District": "...",
        "City": "...",
        "Number_of_people": "...",
        "Price": "...",
        "Rating": "...",
        "Description": "..."
    }}

    Ensure that the "Type" is one of the following: "Hotel," "Restaurant," or "TouristAttraction." If one or two types are mentioned, return only those. If none are mentioned, include all three types.

    For any information not specified in the travel request, return `null`. Ensure that the JSON result is strictly valid JSON, with no extra text, comments, or parentheses.

    The "District" not include "quận" or "huyện". The "City" not include "thành phố" or "tỉnh".

    The "Price" should be an integer. The "Rating" should be a float.
    """
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm

Run nếu dùng LLama 3

In [ ]:
query = """
Gợi ý cho tôi 1 khách sạn ở Cầu Giấy, Hà Nội với chất lượng dịch vụ cao cấp!
"""
response = chain.invoke(query)
cleaned_json_str = re.search(r'\{.*?\}', response, re.DOTALL).group(0)
result_dict = json.loads(cleaned_json_str)
result_dict

Run nếu dùng Gemini


In [ ]:
query = """
Gợi ý cho tôi 1 khách sạn ở Cầu Giấy, Hà Nội với chất lượng dịch vụ cao cấp!
"""
response = chain.invoke(query)
response_gemini = str(response.content)
cleaned_json_str = re.search(r'\{.*?\}', response_gemini, re.DOTALL).group(0)
result_dict = json.loads(cleaned_json_str)
result_dict

In [ ]:
# Gợi ý cho tôi khách sạn ở Cầu Giấy, Hà Nội chất lượng 5 sao giành cho 2 người giá khoảng 10 triệu đồng với chất lượng dịch vụ cao cấp!
query = input("Nhập câu truy vấn của bạn: ")

In [ ]:
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

# Đặt schema hiện tại
cur.execute("SET search_path TO travel_database, public;")

# Tạo vector cho mô tả truy vấn


# Thành phố cần tìm
response = chain.invoke(query)
cleaned_json_str = re.search(r'\{.*?\}', response, re.DOTALL).group(0)
result_dict = json.loads(cleaned_json_str)


embedding_query = model.encode(result_dict['Description'])
embedding_query_list = embedding_query.tolist()

# Kiểm tra xem bảng Hotel có tồn tại không
cur.execute("SELECT * FROM information_schema.tables WHERE table_name = 'hotel';")
if not cur.fetchone():
    print("Bảng 'Hotel' không tồn tại trong schema 'travel_database'.")
else:
    # Thực hiện truy vấn để tìm 2 khách sạn có độ tương đồng cao nhất
    cur.execute("""
        WITH query_embedding AS (
            SELECT %s::vector(768) AS query_vector
        ),
        similarity_scores AS (
            SELECT
                h.hotel_id,
                h.name,
                h.address,
                h.rating,
                h.description,
                (1 - (h.embedding_description <=> query_vector)) AS similarity
            FROM
                Hotel h
                CROSS JOIN query_embedding
            WHERE
                (h.address).city = %s
            ORDER BY
                similarity DESC
            LIMIT 2
        )
        -- Chọn 2 khách sạn có độ tương đồng cao nhất
        SELECT
            hotel_id,
            name,
            address,
            rating,
            description,
            similarity
        FROM
            similarity_scores;
    """, (embedding_query_list, result_dict['City']))

    # Lấy kết quả và in ra
    rows = cur.fetchall()

    print(f"Yêu cầu người dùng: {query}")
    for row in rows:
        print(f"HotelID: {row[0]}")
        print(f"Name: {row[1]}")
        print(f"Address: {row[2]}")
        print(f"Rating: {row[3]}")
        print(f"Description: {row[4]}")
        print(f"Similarity: {row[5]}")
        print("-" * 40)

    # print(result_dict)
# Đóng kết nối
cur.close()
conn.close()

In [ ]:
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

# Đặt schema hiện tại
cur.execute("SET search_path TO travel_database, public;")

# Tạo vector cho mô tả truy vấn


# Thành phố cần tìm
response = chain.invoke(query)
response_gemini = str(response.content)
cleaned_json_str = re.search(r'\{.*?\}', response_gemini, re.DOTALL).group(0)
result_dict = json.loads(cleaned_json_str)


embedding_query = model.encode(result_dict['Description'])
embedding_query_list = embedding_query.tolist()

# Kiểm tra xem bảng Hotel có tồn tại không
cur.execute("SELECT * FROM information_schema.tables WHERE table_name = 'hotel';")
if not cur.fetchone():
    print("Bảng 'Hotel' không tồn tại trong schema 'travel_database'.")
else:
    # Thực hiện truy vấn để tìm 2 khách sạn có độ tương đồng cao nhất
    cur.execute("""
        WITH query_embedding AS (
            SELECT %s::vector(768) AS query_vector
        ),
        similarity_scores AS (
            SELECT
                h.hotel_id,
                h.name,
                h.address,
                h.rating,
                h.description,
                (1 - (h.embedding_description <=> query_vector)) AS similarity
            FROM
                Hotel h
                CROSS JOIN query_embedding
            WHERE
                (h.address).city = %s
            ORDER BY
                similarity DESC
            LIMIT 2
        )
        -- Chọn 2 khách sạn có độ tương đồng cao nhất
        SELECT
            hotel_id,
            name,
            address,
            rating,
            description,
            similarity
        FROM
            similarity_scores;
    """, (embedding_query_list, result_dict['City']))

    # Lấy kết quả và in ra
    rows = cur.fetchall()

    print(f"Yêu cầu người dùng: {query}")
    for row in rows:
        print(f"HotelID: {row[0]}")
        print(f"Name: {row[1]}")
        print(f"Address: {row[2]}")
        print(f"Rating: {row[3]}")
        print(f"Description: {row[4]}")
        print(f"Similarity: {row[5]}")
        print("-" * 40)

    print(result_dict)
# Đóng kết nối
cur.close()
conn.close()